In [ ]:
import json
from pathlib import Path
import pandas as pd

data_dir = Path("/Users/user/Documents/pythonmain/YOLOX/datasets/SkyFusion")

annotation_sub_dir = "annotations"

In [ ]:
def filter_annotation_class(json_anns, wanted_class = 0):
    annotations = pd.DataFrame(json_anns["annotations"])
    images = pd.DataFrame(json_anns["images"])
    wanted_annotations = annotations[annotations["category_id"] == wanted_class]
    wanted_images = images.merge(annotations["image_id"].drop_duplicates(), 
                                 left_on="id", right_on="image_id",
                                 how = "inner").drop("image_id", axis = 1)
    json_anns["images"] = wanted_images.to_dict(orient="records")
    json_anns["annotations"] = wanted_annotations.to_dict(orient="records")
    json_anns["categories"] = [c for c in json_anns["categories"] if c["id"] == wanted_class]
    return json_anns, wanted_images["file_name"].values.tolist()

def filter_dataset(json_ann_file: Path, image_dir: Path, wanted_class: int = 0):
    with open(json_ann_file, "r") as f:
        anns = json.load(f)
    anns, want_ims = filter_annotation_class(anns, wanted_class)

    for f in image_dir.iterdir():
        if f.name not in want_ims:
            f.unlink()

    with open(json_ann_file, "w") as f:
        json.dump(anns, f, indent=4)

In [ ]:
filter_dataset(
    data_dir / "annotations/instances_train2017.json",
    data_dir / "train2017",
    0
)

filter_dataset(
    data_dir / "annotations/instances_val2017.json",
    data_dir / "val2017",
    0
)

filter_dataset(
    data_dir / "annotations/instances_test2017.json",
    data_dir / "test2017",
    0
)

In [ ]:
with open(data_dir / "annotations/instances_train2017.json", "r") as f:
    anns = json.load(f)

In [ ]:
pd.DataFrame(anns["images"])